## OCC solution

In [60]:
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
import sys
sys.path.append('.\\..\\..\\Chapter2-Z24-dataset')
from predict import binarize
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, accuracy_score

In [4]:
X_train = np.load("C:\\Users\\amroa\\Documents\\thesis\\X_train.npy")
labels_train = np.load("C:\\Users\\amroa\\Documents\\thesis\\labels_train.npy")
X_test = np.load("C:\\Users\\amroa\\Documents\\thesis\\X_test.npy")
labels_test = np.load("C:\\Users\\amroa\\Documents\\thesis\\labels_test.npy")

In [17]:
# extract the relevant file indices for training and testing separately
test_indices=  np.array([])
train_indices=  np.array([])

for i in range(17):
    random.seed(i)
    nbr_splits = 36 #  recall 612/17 = 36, where 612 = total nbr of epochs, 17 is nbr of scenarios
    test_split = 4
    test_indices_temp = np.array(random.sample(range(nbr_splits), test_split))
    test_indices = np.append(test_indices, i*nbr_splits + test_indices_temp)
    train_indices = np.append(train_indices,  i*nbr_splits+ np.array([x for x in range(nbr_splits) if x not in test_indices_temp]))

print(len(train_indices))

544


In [20]:
train_ctr = 0
test_ctr = 0
data_all = []
labels_all = []
for i in range(612):
    if i in train_indices:
        data_all.append(X_train[train_ctr])
        labels_all.append(labels_train[train_ctr])
        train_ctr += 1
    elif i in test_indices:
        data_all.append(X_test[test_ctr])
        labels_all.append(labels_test[test_ctr])
        test_ctr += 1

In [29]:
y_all = binarize(labels_all) # is the labels of all instances 
data_all = np.array(data_all) # are all the instances

In [62]:
# define hyperparameters and data
features_to_keep = 79
neighbors = 41
y_train = binarize(labels_all)
log_reg = LogisticRegression(C = 12.5) 
y_train = binarize(labels_train)
log_reg.fit(X_train, y_train)
sorted_feature_indices = np.argsort(log_reg.coef_[0])[::-1] # get the indices of the most important features in descending order
indices = sorted_feature_indices[:features_to_keep]
data_all_top = data_all[:, indices] 

In [ ]:
def split_data(healthy_data, test_size, validation_size):
    # Split data into training and testing
    healthy_train, healthy_test = train_test_split(healthy_data, test_size=test_size, random_state=42)
    # Split training data further to get a validation set
    healthy_train, healthy_validation = train_test_split(healthy_train, test_size=validation_size, random_state=42)
    
    return healthy_train, healthy_validation, healthy_test

In [54]:
def evaluate_model(model, healthy_data, unhealthy_validation, unhealthy_test, train_split, tune=False):
    window = int(train_split*healthy_data.shape[0])
    print(f"Using {window} instances")
    model.fit(healthy_data[-window:])
    
    # predict on validation and test data (damaged)
    preds_validation = model.predict(unhealthy_validation)
    preds_test = model.predict(unhealthy_test)
    
    # occ models are unsupervised, they predict +1 for inliers and -1 for outliers
    # invert the predictions because in our case, healthy is 0 and unhealthy is 1
    preds_validation = (preds_validation == -1).astype(int)
    preds_test = (preds_test == -1).astype(int)

    # labels for validation and test sets (since we know they should be 1 for unhealthy)
    unhealthy_validation_labels = np.ones(len(unhealthy_validation))
    unhealthy_test_labels = np.ones(len(unhealthy_test))
    
    # evaluate model
    print("Validation Results:")
    print(classification_report(unhealthy_validation_labels, preds_validation))
    print("Accuracy:", accuracy_score(unhealthy_validation_labels, preds_validation))
    
    print("Test Results:")
    print(classification_report(unhealthy_test_labels, preds_test))
    print("Accuracy:", accuracy_score(unhealthy_test_labels, preds_test))


In [63]:
# Split the data into healthy and unhealthy instances
healthy_data = data_all_top[y_all == 0]
unhealthy_data = data_all_top[y_all == 1]

unhealthy_validation, unhealthy_test = train_test_split(
    unhealthy_data, test_size=0.8, random_state=42
)

# crwate models
svm_model = OneClassSVM(kernel='linear', gamma='auto')
isolation_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
lof_model = LocalOutlierFactor(n_neighbors=20, novelty=True)

In [64]:
healthy_data.shape, unhealthy_data.shape

((288, 79), (324, 79))

In [65]:
# evaluate all
print("Evaluating One-Class SVM...")
for el in np.arange(0.05, 1.0, 0.1):
    evaluate_model(svm_model, healthy_data, unhealthy_validation, unhealthy_test, el)

#print("\nEvaluating Isolation Forest...")
#evaluate_model(isolation_forest, healthy_data, unhealthy_validation, unhealthy_test)

#print("\nEvaluating Local Outlier Factor...")
#evaluate_model(lof_model, healthy_data, unhealthy_validation, unhealthy_test)

Evaluating One-Class SVM...
Using 14 instances
Validation Results:
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        64

    accuracy                           1.00        64
   macro avg       1.00      1.00      1.00        64
weighted avg       1.00      1.00      1.00        64

Accuracy: 1.0
Test Results:
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       260

    accuracy                           1.00       260
   macro avg       1.00      1.00      1.00       260
weighted avg       1.00      1.00      1.00       260

Accuracy: 1.0
Using 43 instances
Validation Results:
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        64

    accuracy                           1.00        64
   macro avg       1.00      1.00      1.00        64
weighted avg       1.00      1.00      1.00        64

Accuracy: 1.0
Test Results:
     

c:\Users\amroa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amroa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amroa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [52]:
np.sort(np.hstack((test_indices, train_indices))).astype(int)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18